# Project 1

In [49]:
# Import libaries here

import pandas as pd
import requests
import datetime as dt
import time
from bs4 import BeautifulSoup

In [27]:
url_1 = 'https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance'
url_2 = 'https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge'

In [28]:
res_1 = requests.get(url_1)
res_1.status_code

200

In [30]:
res_2 = requests.get(url_2)
res_2.status_code

200

In [32]:
json_1 = res_1.json()
#json_1

In [34]:
json_2 = res_2.json()
#json_2

In [35]:
# Checking parameters in 1 post to see what features I want to keep
json_1['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'Phatferd',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_54hdd',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1627597770,
 'domain': 'self.MaliciousCompliance',
 'full_link': 'https://www.reddit.com/r/MaliciousCompliance/comments/ou7wra/how_i_got_out_of_apartments_lease/',
 'gildings': {},
 'id': 'ou7wra',
 'is_created_from_ads_ui': False,
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': True,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '#373c3f',
 'link_flair_css_class': 'sf',
 'link_flair_richtext': [],
 'link_flair_template_id': '8d5a9196-07ff-11e7-96b0-0e3f84158fec',
 'link_flair_text': 'S',


In [71]:
#REFERENCE: pushshift_demo lesson

def query_pushshift(subreddit, kind = 'submission', day_window = 30, n = 60):
    SUBFIELDS = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']
    
    # establish base url and stem
    BASE_URL = f"https://api.pushshift.io/reddit/search/{kind}" # also known as the "API endpoint" 
    stem = f"{BASE_URL}?subreddit={subreddit}&size=500" # always pulling max of 500
    
    # instantiate empty list for temp storage
    posts = []
    
    # implement for loop with `time.sleep(2)`
    for i in range(1, n + 1):
        URL = "{}&after={}d".format(stem, day_window * i)
        print("Querying from: " + URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        posts.append(df)
        time.sleep(2)
    
    # pd.concat storage list
    full = pd.concat(posts, sort=False)
    
    # if submission
    if kind == "submission":
        # select desired columns
        full = full[SUBFIELDS]
        # drop duplicates
        full.drop_duplicates(inplace = True)
        # select `is_self` == True
        full = full.loc[full['is_self'] == True]

    # create `timestamp` column
    full['timestamp'] = full["created_utc"].map(dt.date.fromtimestamp)
    
    print("Query Complete!")    
    return full 

In [74]:
results1 = query_pushshift('MaliciousCompliance')

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=150d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=180d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=210d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=MaliciousCompliance&size=500&after=240d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=Maliciou

In [72]:
results2 = query_pushshift('ProRevenge')

Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=30d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=60d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=90d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=120d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=150d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=180d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=210d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=240d
Querying from: https://api.pushshift.io/reddit/search/submission?subreddit=ProRevenge&size=500&after=270d
Querying from: https://api.pushshift.io/reddit/se

In [80]:
results1.shape, results2.shape

((4309, 9), (5914, 9))

In [78]:
results1.tail()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
11,"the more I deal with the phone company, the mo...",I used to know someone who had that on a butto...,MaliciousCompliance,1474237291,MKEgal,25,240,True,2016-09-18
12,[NSFW] Dumbarse of the Day - Me,[removed],MaliciousCompliance,1474287658,MexicanSpaceProgram,29,36,True,2016-09-19
13,Of course I can explain it to you,"There's an old saying in the military, ""If you...",MaliciousCompliance,1474397336,FishinMike,85,988,True,2016-09-20
14,If you insist...,So... $firm has a huge contract with $institut...,MaliciousCompliance,1474432195,an-3,72,292,True,2016-09-20
15,"[NSFW] Fuck you, post office",While I have a level of contempt for the knuck...,MaliciousCompliance,1474442176,MexicanSpaceProgram,45,97,True,2016-09-21


In [79]:
results2.tail()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
91,[Unexpected Update] Revenge is a dish best ser...,"So, after [this post,](https://www.reddit.com/...",ProRevenge,1474479776,batkevn,37,579,True,2016-09-21
92,Comcast ripped me off so I developed a 0-day X...,[removed],ProRevenge,1474532022,zr0day,0,0,True,2016-09-22
93,The Last Straw,[removed],ProRevenge,1474586673,[deleted],6,0,True,2016-09-22
94,(Acne I guess ? ) anyone who can help please,[deleted],ProRevenge,1474593774,[deleted],0,1,True,2016-09-22
95,"Deface my property, I'll go Ezekiel 25:17 on y...",I'll start off by saying that high school was ...,ProRevenge,1474600389,crookshanks777,203,1708,True,2016-09-22


In [81]:
results1.isnull().sum()

title            0
selftext        40
subreddit        0
created_utc      0
author           0
num_comments     0
score            0
is_self          0
timestamp        0
dtype: int64

In [82]:
results2.isnull().sum()

title             0
selftext        127
subreddit         0
created_utc       0
author            0
num_comments      0
score             0
is_self           0
timestamp         0
dtype: int64

In [85]:
# Nulls are where the posts may only have a title and no body text - I don't want to delete since titles are useful
# Fill NaNs with 'none'

results1['selftext'] = results1['selftext'].fillna('none')
results2['selftext'] = results2['selftext'].fillna('none')